# Import Necessary Packages

In [1]:
import pandas as pd
import numpy as np
import warnings
import copy as cp

from datetime import datetime, timedelta

warnings.filterwarnings("ignore")

# Dimensions

## Country

In [2]:
country_list = ["Bangladesh", "Canada", "Chad", "Guinea", "Indonesia", "Mexico", "Philippines", "Togo", "United States"]

# Create a dataframe to hold the country dimension data
# Has a surrogate attribute "year" to help us combine the dimensions in the fact table
country_dimension_df = pd.DataFrame(
    columns=["Country_key", "Surrogate_year", "Name", "Region", "Continent", "Currency", "Capital", "Total_population","Population_growth_percent", "Total_labour_force", "Birth_rate", "Death_rate",  "GNI_per_capita", "Income_group"]
)

# Values found by looking up countries on the internet
dict_capital = {
    'Bangladesh': 'Dhaka',
    'Canada': 'Ottawa',
    'Chad': 'N\'Djamena',
    'Guinea': 'Conakry',
    'Indonesia': 'Jakarta',
    'Mexico': 'Mexico City',
    'Philippines': 'Manila',
    'Togo': 'Lomé',
    'United States': 'Washington, D.C.'
}
# Values found by looking up countries on the internet
dict_continent = {
    'Bangladesh': 'Asia',
    'Canada': 'North America',
    'Chad': 'Africa',
    'Guinea': 'Africa',
    'Indonesia': 'Asia',
    'Mexico': 'North America',
    'Philippines': 'Asia',
    'Togo': 'Africa',
    'United States': 'North America'
}

# CSV file of HNP country data
raw_geo_data = pd.read_csv('raw_data/HNP_CountryData.csv')
# CSV file of HNP country dimension statistics
raw_att_data = pd.read_csv('raw_data/HNP_CountryAttributes.csv')
# Get rid of two columns we don't need
raw_att_data = raw_att_data.drop(
    ['Series Code', 'Country Code'], axis=1
)
# Rename the columns for easier access
raw_att_data = raw_att_data.rename(
    columns={
        'Series Name': 'Series',
        'Country Name': 'Country',
        '2005 [YR2005]': 'y2005',
        '2006 [YR2006]': 'y2006',
        '2007 [YR2007]': 'y2007',
        '2008 [YR2008]': 'y2008',
        '2009 [YR2009]': 'y2009',
        '2010 [YR2010]': 'y2010',
        '2011 [YR2011]': 'y2011',
        '2012 [YR2012]': 'y2012',
        '2013 [YR2013]': 'y2013',
        '2014 [YR2014]': 'y2014',
        '2015 [YR2015]': 'y2015',
        '2016 [YR2016]': 'y2016',
        '2017 [YR2017]': 'y2017',
        '2018 [YR2018]': 'y2018',
        '2019 [YR2019]': 'y2019',
        '2020 [YR2020]': 'y2020'
    }
)

# Start key at 1000
x = 1000
for country in country_list:
    country_row = {}
    # Add the country attributes that don't change
    country_row['Name'] = country
    country_row['Region'] = raw_geo_data.loc[raw_geo_data['Table Name'] == country, 'Region'].values[0]
    country_row['Continent'] = dict_continent[country]
    country_row['Currency'] = raw_geo_data.loc[raw_geo_data['Table Name'] == country, 'Currency Unit'].values[0]
    country_row['Capital'] = dict_capital[country]
    country_row['Income_group'] = raw_geo_data.loc[raw_geo_data['Table Name'] == country, 'Income Group'].values[0]

    specific_row = cp.deepcopy(country_row)
    # Add the country attributes that change every year
    for year in range(2005,2021):
        specific_row['Country_key'] = x
        x = x + 1
        specific_row['Surrogate_year'] = year
        specific_row['Total_population'] = raw_att_data.loc[((raw_att_data.Series == 'Population, total') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        specific_row['Population_growth_percent'] = raw_att_data.loc[((raw_att_data.Series == 'Population growth (annual %)') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        specific_row['Total_labour_force'] = raw_att_data.loc[((raw_att_data.Series == 'Labor force, total') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        specific_row['Birth_rate'] = raw_att_data.loc[((raw_att_data.Series == 'Birth rate, crude (per 1,000 people)') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        specific_row['Death_rate'] = raw_att_data.loc[((raw_att_data.Series == 'Death rate, crude (per 1,000 people)') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        specific_row['GNI_per_capita'] = raw_att_data.loc[((raw_att_data.Series == 'GNI per capita, Atlas method (current US$)') & (raw_att_data.Country == country)), 'y' + str(year)].values[0]
        # Adding the row into the country dimension dataframe
        country_dimension_df = country_dimension_df.append(pd.DataFrame([specific_row]), ignore_index=True)

country_dimension_df.head(5)

,Country_key,Surrogate_year,Name,Region,Continent,Currency,Capital,Total_population,Population_growth_percent,Total_labour_force,Birth_rate,Death_rate,GNI_per_capita,Income_group
0,1000,2005,Bangladesh,South Asia,Asia,Bangladeshi taka,Dhaka,139035505.0,1.484747,53123178.0,24.053,6.205,550.0,Lower middle income
1,1001,2006,Bangladesh,South Asia,Asia,Bangladeshi taka,Dhaka,140921154.0,1.347121,54083636.0,23.384,6.092,570.0,Lower middle income
2,1002,2007,Bangladesh,South Asia,Asia,Bangladeshi taka,Dhaka,142660381.0,1.226631,54954286.0,22.747,5.987,610.0,Lower middle income
3,1003,2008,Bangladesh,South Asia,Asia,Bangladeshi taka,Dhaka,144304164.0,1.145647,55799460.0,22.153,5.892,660.0,Lower middle income
4,1004,2009,Bangladesh,South Asia,Asia,Bangladeshi taka,Dhaka,145924795.0,1.116806,56639145.0,21.605,5.807,730.0,Lower middle income


## Month

In [3]:
month_values = ["January","February","March","April","May","June","July","August","September","October","November","December"]
first_quarter = ["January","February","March"]
secound_quarter = ["April","May","June"]
third_quarter = ["July","August","September"]
forth_quarter = ["October","November","December"]
year_values = list(range(2005,2021))#this gives a list of values starting at 2005 to 2020
decade_values = [1,2]

#create list of dicts that represent each row
key_value = 2000
month_row_list = []
for year in year_values:
    for month in month_values:

        key_value += 1
        mount_row = {}
        mount_row["Month_Key"] = key_value
        mount_row["Month"] = month
        mount_row["Year"] = year

        if month in first_quarter:
            mount_row["Quarter"] = 1
        elif month in secound_quarter:
            mount_row["Quarter"] = 2
        elif month in third_quarter:
            mount_row["Quarter"] = 3
        else:
            mount_row["Quarter"] = 4
        
        if year <= 2010:
            mount_row["Decade"] = 1
        else:
            mount_row["Decade"] = 2

        month_row_list.append(mount_row)

month_df = pd.DataFrame(month_row_list)
print(month_df.head())

   Month_Key     Month  Year  Quarter  Decade
0       2001   January  2005        1       1
1       2002  February  2005        1       1
2       2003     March  2005        1       1
3       2004     April  2005        2       1
4       2005       May  2005        2       1


## Education

In [4]:
hnp_data = pd.read_csv("raw_data/education.csv")
hnp_data.head()

,SeriesName,SeriesCode,CountryName,CountryCode,2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,Africa Eastern and Southern,AFE,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
1,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,Africa Western and Central,AFW,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..,..
2,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,Arab World,ARB,58.8121681213379,58.6921691894531,59.1082191467285,60.4463195800781,58.9698181152344,60.2376518249512,62.4384498596191,66.0942764282227,65.2387313842773,67.7367477416992,65.5670394897461,67.3514862060547,68.6543579101563,65.5303192138672,65.8576126098633,66.1968994140625
3,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,Caribbean small states,CSS,91.4608535766602,91.6320877075195,92.3097915649414,92.5576705932617,92.7684631347656,92.9917678833008,93.1499099731445,93.0394897460938,92.8167495727539,92.9547271728516,93.0601425170898,..,..,..,..,..
4,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,Central Europe and the Baltics,CEB,98.1644287109375,98.184440612793,98.2189102172852,98.2749710083008,98.5478668212891,98.5772323608398,98.6045684814453,98.6247100830078,98.6415405273438,98.6618728637695,98.6808471679688,98.7023162841797,..,..,..,..


In [5]:

education_df = pd.DataFrame(
    columns = ["surr_year","surr_country","education_key", "literacy_rate_percent_m_15_24","literacy_rate_percent_15_24","literacy_rate_percent_f_15_plus","literacy_rate_percent_m_15_plus","literacy_percent_15_plus","primary_completion_rate_percent_f","primary_completion_rate_percent_m","primary_completion_rate_percent","primary_enroll_rate_percent_gross","pimary_enroll_rate_percent_gross_f","pimary_enroll_rate_percent_gross_m","primary_enroll_rate_percent_net","primary_enroll_rate_percent_net_f","primary_enroll_rate_percent_net_m","secondary_enroll_rate_percent_gross","secondary_enroll_rate_percent_gross_f","secondary_enroll_rate_percent_gross_m","secondary_enroll_rate_percent_net","secondary_enroll_rate_percent_net_f","secondary_enroll_rate_percent_net_m","tertiary_enroll_rate_percent_gross","spending_education_percent_gdp"]
)



In [6]:
country_list = ["Bangladesh", "Canada", "Chad", "Guinea", "Indonesia", "Mexico", "Philippines", "Togo", "United States"]
year_list = ["2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

ed_country_dim_data = {}
for country in country_list:
    ed_country_dim_data[country]={}
    for year in year_list:
        ed_country_dim_data[country][year] = {}
counter = 0  

year_idx = {}
idx = 4
for year in year_list:
    year_idx[year] = idx
    idx=idx+1
    
def populate_dim_data(dim_data,country,column,row): 
    empty_year = []
    total = 0
    counter = 0
    for key, value in year_idx.items():
        if row[value] == "..":
            empty_year.append(key)
        else:
            dim_data[country][key][column] = row[value]
            total = total + float(row[value])
            if row[value] != 0:
                counter = counter+1
    for key, value in year_idx.items():
        if key in empty_year:
            if row[value] == ".." and counter != 0:
                dim_data[country][key][column] = str(total/counter)
            elif row[value] == "..":
                dim_data[country][key][column] = -1
 
        
        
for country in country_list:
    for year in year_list:
        ed_country_dim_data[country][year]["education_key"]= str(counter+3000)
        ed_country_dim_data[country][year]["surr_country"]= country
        ed_country_dim_data[country][year]["surr_year"]= year
        counter=counter+1


for row in hnp_data.values.tolist():
    if row[2] in country_list:
        if row[0] == "Literacy rate, adult female (% of females ages 15 and above)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["literacy_rate_percent_f_15_plus"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"literacy_rate_percent_f_15_plus",row)
        elif row[0] == "Literacy rate, adult male (% of males ages 15 and above)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["literacy_rate_percent_m_15_plus"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"literacy_rate_percent_m_15_plus",row)
        elif row[0] == "Literacy rate, adult total (% of people ages 15 and above)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["literacy_percent_15_plus"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"literacy_percent_15_plus",row)
        elif row[0] == "Literacy rate, youth total (% of people ages 15-24)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["literacy_rate_percent_15_24"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"literacy_rate_percent_15_24",row)
        elif row[0] == "Literacy rate, youth male (% of males ages 15-24)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["literacy_rate_percent_m_15_24"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"literacy_rate_percent_m_15_24",row)
        elif row[0] == "Primary completion rate, female (% of relevant age group)":
            if (row[2]=="Canada"):
                for year in year_list:
                    ed_country_dim_data[row[2]][year]["primary_completion_rate_percent_f"] = 100
            else:
                populate_dim_data(ed_country_dim_data,row[2],"primary_completion_rate_percent_f",row)
        elif row[0] == "Primary completion rate, male (% of relevant age group)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_completion_rate_percent_m",row)
        elif row[0] == "Primary completion rate, total (% of relevant age group)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_completion_rate_percent",row)
        elif row[0] == "School enrollment, primary (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_enroll_rate_percent_gross",row)
        elif row[0] == "School enrollment, primary, female (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"pimary_enroll_rate_percent_gross_f",row)
        elif row[0] == "School enrollment, primary, male (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"pimary_enroll_rate_percent_gross_m",row)
        elif row[0] == "School enrollment, primary (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_enroll_rate_percent_net",row)
        elif row[0] == "School enrollment, primary, female (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_enroll_rate_percent_net_f",row)
        elif row[0] == "School enrollment, primary, male (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"primary_enroll_rate_percent_net_m",row)
        elif row[0] == "School enrollment, secondary (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_gross",row)
        elif row[0] == "School enrollment, secondary, female (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_gross_f",row)
        elif row[0] == "School enrollment, secondary, male (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_gross_m",row)
        elif row[0] == "School enrollment, secondary (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_net",row)
        elif row[0] == "School enrollment, secondary, female (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_net_f",row)
        elif row[0] == "School enrollment, secondary, male (% net)":
            populate_dim_data(ed_country_dim_data,row[2],"secondary_enroll_rate_percent_net_m",row)
        elif row[0] == "School enrollment, tertiary (% gross)":
            populate_dim_data(ed_country_dim_data,row[2],"tertiary_enroll_rate_percent_gross",row)
        elif row[0] == "Public spending on education, total (% of GDP)":
            populate_dim_data(ed_country_dim_data,row[2],"spending_education_percent_gdp",row)


for country in country_list:
    for year in year_list:
        education_df = education_df.append(pd.DataFrame([ed_country_dim_data[country][year]]), ignore_index=True)

In [7]:
education_df

,surr_year,surr_country,education_key,literacy_rate_percent_m_15_24,literacy_rate_percent_15_24,literacy_rate_percent_f_15_plus,literacy_rate_percent_m_15_plus,literacy_percent_15_plus,primary_completion_rate_percent_f,primary_completion_rate_percent_m,...,primary_enroll_rate_percent_net_f,primary_enroll_rate_percent_net_m,secondary_enroll_rate_percent_gross,secondary_enroll_rate_percent_gross_f,secondary_enroll_rate_percent_gross_m,secondary_enroll_rate_percent_net,secondary_enroll_rate_percent_net_f,secondary_enroll_rate_percent_net_m,tertiary_enroll_rate_percent_gross,spending_education_percent_gdp
0,2005,Bangladesh,3000,83.90767184170811,85.86140233820134,62.37429705533116,68.52192237160425,65.42699536410247,68.8696670532227,63.8693199157715,...,93.66587,91.5114,46.9628295898438,48.4769515991211,45.5096092224121,43.94428,45.41747,42.53033,6.44670009613037,1.9404133160909
1,2006,Bangladesh,3001,83.90767184170811,85.86140233820134,62.37429705533116,68.52192237160425,65.42699536410247,67.78083515167236,61.65665721893313,...,93.66587,91.5114,47.5297813415527,48.8530082702637,46.2574310302734,44.60284,45.90001,43.35556,7.36119985580444,2.13128995895386
2,2007,Bangladesh,3002,54.2743301391602,61.8728103637695,43.7408485412598,49.8323287963867,46.6636009216309,67.78083515167236,61.65665721893313,...,93.66587,91.5114,48.3010597229004,49.5662803649902,47.0835113525391,45.41006,46.63704,44.22933,7.93297004699707,2.20221996307373
3,2008,Bangladesh,3003,83.90767184170811,85.86140233820134,62.37429705533116,68.52192237160425,65.42699536410247,61.9892082214355,56.1707000732422,...,93.66587,91.5114,46.2411193847656,49.0108909606934,43.5757598876953,52.81895583333333,55.859675,49.903886666666665,8.88409042358398,2.0545699596405
4,2009,Bangladesh,3004,83.90767184170811,85.86140233820134,62.37429705533116,68.52192237160425,65.42699536410247,68.4999465942383,62.5326614379883,...,93.66587,91.5114,49.9982414245605,51.9128303527832,48.1570014953613,46.3405,48.25183,44.50241,10.8580904006958,1.9394199848175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2016,United States,3139,100,100,100,100,100,98.66200002034525,100.89743296305362,...,94.85693,95.28074,98.7699279785156,98.4747085571289,99.0528335571289,92.18317,92.17723,92.18887,88.8350524902344,-1
140,2017,United States,3140,100,100,100,100,100,97.1660308837891,100.429466247559,...,94.57828,94.6724,98.9523391723633,98.695556640625,99.1985168457031,92.45022,93.03496,91.88962,88.1673889160156,-1
141,2018,United States,3141,100,100,100,100,100,95.4763412475586,104.510192871094,...,93.85715222222224,93.48976555555556,99.2755813598633,98.7960586547852,99.7356567382813,90.22441555555558,91.08291777777778,89.40598666666666,88.2991790771484,-1
142,2019,United States,3142,100,100,100,100,100,103.343627929688,97.7526397705078,...,93.85715222222224,93.48976555555556,100.063430786133,99.189453125,100.902122497559,90.22441555555558,91.08291777777778,89.40598666666666,87.8887100219727,-1


## Health

In [8]:
health_data = pd.read_csv("raw_data/health.csv")
health_data.head()

,Series Name,Series Code,Country Name,Country Code,2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Cause of death, by communicable diseases and m...",SH.DTH.COMM.ZS,Africa Eastern and Southern,AFE,..,..,..,..,..,59.1548866588657,..,..,..,..,52.911433522748,..,..,..,49.0198435796198,..
1,"Cause of death, by communicable diseases and m...",SH.DTH.COMM.ZS,Africa Western and Central,AFW,..,..,..,..,..,65.1492323583742,..,..,..,..,61.8831660993608,..,..,..,59.5038196263206,..
2,"Cause of death, by communicable diseases and m...",SH.DTH.COMM.ZS,Arab World,ARB,..,..,..,..,..,24.1138600513385,..,..,..,..,19.7153918077943,..,..,..,18.818142806916,..
3,"Cause of death, by communicable diseases and m...",SH.DTH.COMM.ZS,Caribbean small states,CSS,..,..,..,..,..,12.5585531384107,..,..,..,..,11.5926265881526,..,..,..,10.8163825211863,..
4,"Cause of death, by communicable diseases and m...",SH.DTH.COMM.ZS,Central Europe and the Baltics,CEB,..,..,..,..,..,3.3345773309974,..,..,..,..,3.91859328252736,..,..,..,4.64425613438535,..


In [9]:
health_df = pd.DataFrame(
    columns = ["surr_year",
               "surr_country",
               "health_key",
               "death_by_communicable_diseases_maternal_prenatal_and_nutrition_conditions_percent",
               "death_by_non_communicable_diseases_percent",
               "people_using_at_least_basic_drinking_water_services_percent",
               "people_using_safely_managed_drinking_water_services",
               "hospital_beds",
               "community_health_workers",
               "nurses_and_midwives",
               "physicians",
               "specialist_surgical_workforce",
               "incidence_of_malaria",
               "intermittent_preventive_treatment_of_malaria_in_pregnancy",
               "use_of_insecticide_treated_bed_nets",
               "children_with_fever_receiving_antimalarial_drugs",
               "malaria_cases_reported",
               "people_practicing_open_defecation_rural",
               "people_practicing_open_defecation_urban",
               "people_practicing_open_defecation",
               "people_using_safely_managed_sanitation_services",
               "mortality_rate_attributed_to_unsafe_water_unsafe_sanitation_and_lack_of_hygiene",
               "current_health_expenditure",
               "current_health_expenditure_per_capita",
               "domestic_general_government_health_expenditure",
               "domestic_general_government_health_expenditure_per_capita",
               "external_health_expenditure_per_capita"
              ]
)

In [10]:
country_list = ["Bangladesh", "Canada", "Chad", "Guinea", "Indonesia", "Mexico", "Philippines", "Togo", "United States"]
year_list = ["2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

health_country_dim_data = {}
for country in country_list:
    health_country_dim_data[country]={}
    for year in year_list:
        health_country_dim_data[country][year] = {}
counter = 0  

year_idx = {}
idx = 4
for year in year_list:
    year_idx[year] = idx
    idx=idx+1
    
for country in country_list:
    for year in year_list:
        health_country_dim_data[country][year]["health_key"]= str(counter+4000)
        health_country_dim_data[country][year]["surr_country"]= country
        health_country_dim_data[country][year]["surr_year"]= year
        counter=counter+1
    

for row in health_data.values.tolist():
    if row[2] in country_list:
        if row[0] == "Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total)":
            populate_dim_data(health_country_dim_data,row[2],"death_by_communicable_diseases_maternal_prenatal_and_nutrition_conditions_percent",row)
        if row[0] == "Cause of death, by non-communicable diseases (% of total)":
            populate_dim_data(health_country_dim_data,row[2],"death_by_non_communicable_diseases_percent",row)
        if row[0] == "People using at least basic drinking water services (% of population)":
            populate_dim_data(health_country_dim_data,row[2],"people_using_at_least_basic_drinking_water_services_percent",row)
        if row[0] == "People using safely managed drinking water services (% of population)":
            populate_dim_data(health_country_dim_data,row[2],"people_using_safely_managed_drinking_water_services",row)
        if row[0] == "Hospital beds (per 1,000 people)":
            populate_dim_data(health_country_dim_data,row[2],"hospital_beds",row)
        if row[0] == "Community health workers (per 1,000 people)":
            populate_dim_data(health_country_dim_data,row[2],"community_health_workers",row)
        if row[0] == "Nurses and midwives (per 1,000 people)":
            populate_dim_data(health_country_dim_data,row[2],"nurses_and_midwives",row)
        if row[0] == "Physicians (per 1,000 people)":
            populate_dim_data(health_country_dim_data,row[2],"physicians",row)
        if row[0] == "Specialist surgical workforce (per 100,000 population)":
            populate_dim_data(health_country_dim_data,row[2],"specialist_surgical_workforce",row)
        if row[0] == "Incidence of malaria (per 1,000 population at risk)":
            populate_dim_data(health_country_dim_data,row[2],"incidence_of_malaria",row)
        if row[0] == "Intermittent preventive treatment (IPT) of malaria in pregnancy (% of pregnant women)":
            populate_dim_data(health_country_dim_data,row[2],"intermittent_preventive_treatment_of_malaria_in_pregnancy",row)
        if row[0] == "Use of insecticide-treated bed nets (% of under-5 population)":
            populate_dim_data(health_country_dim_data,row[2],"use_of_insecticide_treated_bed_nets",row)
        if row[0] == "Children with fever receiving antimalarial drugs (% of children under age 5 with fever)":
            populate_dim_data(health_country_dim_data,row[2],"children_with_fever_receiving_antimalarial_drugs",row)
        if row[0] == "Malaria cases reported":
            populate_dim_data(health_country_dim_data,row[2],"malaria_cases_reported",row)
        if row[0] == "People practicing open defecation (% of population)":
            populate_dim_data(health_country_dim_data,row[2],"people_practicing_open_defecation",row)
        if row[0] == "People practicing open defecation, rural (% of rural population)":
            populate_dim_data(health_country_dim_data,row[2],"people_practicing_open_defecation_rural",row)
        if row[0] == "People practicing open defecation, urban (% of urban population)":
            populate_dim_data(health_country_dim_data,row[2],"people_practicing_open_defecation_urban",row)
        if row[0] == "Mortality rate attributed to unsafe water, unsafe sanitation and lack of hygiene (per 100,000 population)":
            populate_dim_data(health_country_dim_data,row[2],"mortality_rate_attributed_to_unsafe_water_unsafe_sanitation_and_lack_of_hygiene",row)
        if row[0] == "People using safely managed sanitation services (% of population)":
            populate_dim_data(health_country_dim_data,row[2],"people_using_safely_managed_sanitation_services",row)
        if row[0] == "Current health expenditure (% of GDP)":
            populate_dim_data(health_country_dim_data,row[2],"current_health_expenditure",row)
        if row[0] == "Current health expenditure per capita (current US$)":
            populate_dim_data(health_country_dim_data,row[2],"current_health_expenditure_per_capita",row)
        if row[0] == "Domestic general government health expenditure (% of current health expenditure)":
            populate_dim_data(health_country_dim_data,row[2],"domestic_general_government_health_expenditure",row)
        if row[0] == "Domestic general government health expenditure (% of current health expenditure)":
            populate_dim_data(health_country_dim_data,row[2],"domestic_general_government_health_expenditure_per_capita",row)
        if row[0] == "External health expenditure (% of current health expenditure)":
            populate_dim_data(health_country_dim_data,row[2],"external_health_expenditure_per_capita",row)

for country in country_list:
    for year in year_list:
        health_df = health_df.append(pd.DataFrame([health_country_dim_data[country][year]]), ignore_index=True)

In [11]:
health_df

,surr_year,surr_country,health_key,death_by_communicable_diseases_maternal_prenatal_and_nutrition_conditions_percent,death_by_non_communicable_diseases_percent,people_using_at_least_basic_drinking_water_services_percent,people_using_safely_managed_drinking_water_services,hospital_beds,community_health_workers,nurses_and_midwives,...,people_practicing_open_defecation_rural,people_practicing_open_defecation_urban,people_practicing_open_defecation,people_using_safely_managed_sanitation_services,mortality_rate_attributed_to_unsafe_water_unsafe_sanitation_and_lack_of_hygiene,current_health_expenditure,current_health_expenditure_per_capita,domestic_general_government_health_expenditure,domestic_general_government_health_expenditure_per_capita,external_health_expenditure_per_capita
0,2005,Bangladesh,4000,28.132965841189932,64.46357407099283,95.59186057,55.15126174,0.3,0.147,0.2839,...,14.84128002,3.570728429,11.81975742,23.07214054,11.9,2.41363597,11.60705376,22.1964798,22.1964798,9.5910635
1,2006,Bangladesh,4001,28.132965841189932,64.46357407099283,95.76021655,55.11252136,0.612,0.33866666666666667,0.2841,...,13.726466,3.29227913,10.85529088,24.14787637,11.9,2.48439622,12.33577824,23.08750534,23.08750534,8.26798153
2,2007,Bangladesh,4002,28.132965841189932,64.46357407099283,95.92502197,55.07018233,0.612,0.33866666666666667,0.1625,...,12.61165197,3.013829831,9.901515171,25.22124717,11.9,2.55897784,14.31881523,22.87058258,22.87058258,8.19128704
3,2008,Bangladesh,4003,28.132965841189932,64.46357407099283,96.0861324,55.02438402,0.612,0.33866666666666667,0.1623,...,11.49683795,2.735380531,8.958818692,26.29209359,11.9,2.48848343,15.80424309,21.33563423,21.33563423,8.51236057
4,2009,Bangladesh,4004,28.132965841189932,64.46357407099283,96.24343833,54.97528262,0.612,0.33866666666666667,0.1714,...,10.38202393,2.456931232,8.027558194,27.35956418,11.9,2.56097794,17.92318726,20.35685921,20.35685921,9.59041405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2016,United States,4139,5.548857100771174,88.09737707796383,99.53713865,96.59863984,2.77,-1,13.19415,...,0,0,0,98.31176198,0.2,16.84432411,9775,50.80243301,50.80243301,0
140,2017,United States,4140,5.548857100771174,88.09737707796383,99.62581578,96.78265168,2.87,-1,14.5812,...,0,0,0,98.29928582,0.2,16.80583572,10103.09179688,50.6502037,50.6502037,0
141,2018,United States,4141,5.548857100771174,88.09737707796383,99.71312112,96.96518791,2.987692307692307,-1,15.685,...,0,0,0,98.28709462,0.2,16.68710518,10515.32324219,50.77178955,50.77178955,0
142,2019,United States,4142,5.3020598009239,88.1449804799197,99.79904438,97.14624349,2.987692307692307,-1,13.19415,...,0,0,0,98.27522266,0.2,16.76706314,10921.01269531,50.84327316,50.84327316,0


## Quality of Life

In [12]:
qol_data = pd.read_csv("raw_data/HNP_QOL_Attributes.csv")
qol_data.head()

,Series Name,Series Code,Country Name,Country Code,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2005 [YR2005],2011 [YR2011],2010 [YR2010],2009 [YR2009],2008 [YR2008],2007 [YR2007],2006 [YR2006]
0,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Bangladesh,BGD,6.59899997711182,7.58799982070923,7.48199987411499,7.41800022125244,7.35699987411499,6.71000003814697,6.74399995803833,6.75500011444092,8,7.05000019073486,5.54500007629395,4.43900012969971,7.36600017547607,7.10400009155273,6.84800004959106,6.46400022506714
1,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Canada,CAN,6.81699991226196,6.59700012207031,6.38899993896484,6.29199981689453,6.24100017547607,5.85099983215332,5.48899984359741,5.25,9.38599967956543,6.46600008010864,7.00899982452393,7.24300003051758,7.00500011444092,5.65199995040894,5.65299987792969,6.09200000762939
2,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Chad,TCD,0.63400000333786,0.656000018119812,0.675000011920929,0.698000013828278,0.726000010967255,0.744000017642975,0.759999990463257,0.768999993801117,1.29299998283386,0.495000004768372,0.620000004768372,0.592000007629395,0.578000009059906,0.560000002384186,0.541000008583069,0.523000001907349
3,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Guinea,GIN,4.42199993133545,4.52899980545044,4.62099981307983,4.71000003814697,4.7350001335144,4.82999992370605,4.95800018310547,5.05700016021729,6.14400005340576,3.82200002670288,4.33500003814697,4.25299978256226,4.21199989318848,4.07999992370605,3.97000002861023,3.92400002479553
4,"Unemployment, female (% of female labor force)",SL.UEM.TOTL.FE.ZS,Indonesia,IDN,4.78599977493286,4.27199983596802,3.88199996948242,4.4229998588562,3.86899995803833,3.59899997711182,4.18100023269653,3.34800004959106,3.77800011634827,10.039999961853,5.56400012969971,6.38800001144409,6.70499992370605,8.10400009155273,9.9040002822876,9.2790002822876


In [13]:
qol_df = pd.DataFrame(
    columns = ["surr_country",
               "surr_year",
               "qol_key",
               "unemployment_female",
               "unemployment_male",
               "unemployment_total",
               "people_basic_handwashing_facilities",
               "people_basic_handwashing_facilities_rural",
               "people_basic_handwashing_facilities_urban",
               "people_using_at_least_basic_sanitation_services",
               "people_using_at_least_basic_sanitation_services_rural",
               "people_using_at_least_basic_sanitation_services_urban",
               "people_using_at_least_basic_drinking_water_services",
               "people_using_at_least_basic_drinking_water_services_rural",
               "people_using_at_least_basic_drinking_water_services_urban" 
              ]
)

In [14]:
qol_country_dim_data = {}
for country in country_list:
    qol_country_dim_data[country]={}
    for year in year_list:
        qol_country_dim_data[country][year] = {}
counter = 0  

year_idx = {}
idx = 4
for year in year_list:
    year_idx[year] = idx
    idx=idx+1
    
for country in country_list:
    for year in year_list:
        qol_country_dim_data[country][year]["qol_key"]= str(counter+5000)
        qol_country_dim_data[country][year]["surr_country"]= country
        qol_country_dim_data[country][year]["surr_year"]= year
        counter=counter+1
    
for row in qol_data.values.tolist():
    if row[2] in country_list:
        if row[0] == "Unemployment, female (% of female labor force)":
            populate_dim_data(qol_country_dim_data,row[2],"unemployment_female",row)
        elif row[0] == "Unemployment, male (% of male labor force)":
            populate_dim_data(qol_country_dim_data,row[2],"unemployment_male",row)
        elif row[0] == "Unemployment, total (% of total labor force)":
            populate_dim_data(qol_country_dim_data,row[2],"unemployment_total",row)
        elif row[0] == "People with basic handwashing facilities including soap and water (% of population)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    qol_country_dim_data[row[2]][year]["people_basic_handwashing_facilities"] = 100
            else:
                populate_dim_data(qol_country_dim_data,row[2],"people_basic_handwashing_facilities",row)
        elif row[0] == "People with basic handwashing facilities including soap and water, rural (% of rural population)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    qol_country_dim_data[row[2]][year]["people_basic_handwashing_facilities_rural"] = 100
            else:
                populate_dim_data(qol_country_dim_data,row[2],"people_basic_handwashing_facilities_rural",row)
        elif row[0] == "People with basic handwashing facilities including soap and water, urban (% of urban population)":
            if (row[2]=="Canada" or row[2]=="United States"):
                for year in year_list:
                    qol_country_dim_data[row[2]][year]["people_basic_handwashing_facilities_urban"] = 100
            else:
                populate_dim_data(qol_country_dim_data,row[2],"people_basic_handwashing_facilities_urban",row)
        elif row[0] == "People using at least basic sanitation services (% of population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_sanitation_services",row)
        elif row[0] == "People using at least basic sanitation services, rural (% of rural population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_sanitation_services_rural",row)
        elif row[0] == "People using at least basic sanitation services, urban (% of urban population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_sanitation_services_urban",row)
        elif row[0] == "People using at least basic drinking water services (% of population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_drinking_water_services",row)
        elif row[0] == "People using at least basic drinking water services, rural (% of rural population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_drinking_water_services_rural",row)
        elif row[0] == "People using at least basic drinking water services, urban (% of urban population)":
            populate_dim_data(qol_country_dim_data,row[2],"people_using_at_least_basic_drinking_water_services_urban",row)


for country in country_list:
    for year in year_list:
        qol_df = qol_df.append(pd.DataFrame([qol_country_dim_data[country][year]]), ignore_index=True)



In [15]:
qol_df

,surr_country,surr_year,qol_key,unemployment_female,unemployment_male,unemployment_total,people_basic_handwashing_facilities,people_basic_handwashing_facilities_rural,people_basic_handwashing_facilities_urban,people_using_at_least_basic_sanitation_services,people_using_at_least_basic_sanitation_services_rural,people_using_at_least_basic_sanitation_services_urban,people_using_at_least_basic_drinking_water_services,people_using_at_least_basic_drinking_water_services_rural,people_using_at_least_basic_drinking_water_services_urban
0,Bangladesh,2005,5000,6.59899997711182,3.1949999332428,4.11899995803833,31.43384069,23.04450233,49.26692431,42.1145323,39.00338912,NaN,96.69158904,96.11044423,97.926927
1,Bangladesh,2006,5001,7.58799982070923,3.2260000705719,4.42999982833862,34.90063361,26.89396136,51.33286319,43.65543619,40.92201562,NaN,96.83255493,96.33115251,97.86159904
2,Bangladesh,2007,5002,7.48199987411499,3.17400002479553,4.38399982452393,38.34090253,30.74342039,53.39880206,45.18826042,42.86451471,NaN,96.96925348,96.55200819,97.7962166
3,Bangladesh,2008,5003,7.41800022125244,3.14599990844727,4.36600017547607,41.75359784,34.59287942,55.46474094,46.71185422,44.83088639,NaN,97.10160247,96.77301128,97.73077968
4,Bangladesh,2009,5004,7.35699987411499,3.11800003051758,4.34999990463257,45.13910117,38.44233845,57.53067982,48.22553857,46.82113066,NaN,97.22961308,96.99416177,97.66528829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,United States,2016,5139,8.60999965667725,10.5080003738403,9.63000011444092,100,100,100,99.86076619,99.90014735,NaN,98.97895208,95.88944969,99.71441514
140,United States,2017,5140,8.05900001525879,10.2760000228882,9.25,100,100,100,99.88081511,99.9963302,NaN,98.88203953,95.51131593,99.69304221
141,United States,2018,5141,5.41400003433228,6.09299993515015,5.78000020980835,100,100,100,99.8810546,99.9963302,NaN,98.87501291,95.51131593,99.69304221
142,United States,2019,5142,4.49599981307983,4.72499990463257,4.61999988555908,100,100,100,99.88129549,99.9963302,NaN,98.86794461,95.51131593,99.69304221


## Population

In [16]:
pop_data = pd.read_csv("raw_data/population.csv")
pop_data.head()

,Series Name,Series Code,Country Name,Country Code,2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Africa Eastern and Southern,AFE,39.3744865570595,39.0949663872957,38.7900640436348,38.4537080478074,38.082601108182,37.6770056265627,37.2375573920502,36.7703817223938,36.2848977616519,35.7916936767915,35.2997529119494,34.8207677118531,34.3577466585061,33.9130337240831,33.4912691346873,..
1,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Africa Western and Central,AFW,42.0670961698889,41.8103264303877,41.5350946094575,41.2430013355936,40.9332733533447,40.6015078394124,40.2415934350541,39.852581401361,39.435067467067,38.993875039074,38.5357221779267,38.0694807661362,37.6041829377417,37.1484648274377,36.7085358321222,..
2,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Arab World,ARB,27.5377110975321,27.4772787470601,27.4607245785768,27.4629959320729,27.4960000122321,27.5134349302764,27.4848792443032,27.3893380000754,27.211517136518,26.9408446681938,26.5769915732765,26.1346579650608,25.647683889047,25.1471006437938,24.6506238242473,..
3,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Caribbean small states,CSS,18.6742164291779,18.3902877732859,18.1995377705877,17.9816630724178,17.7684166386154,17.5120687464599,17.3462244942164,17.1386750164885,16.9343698066029,16.6552625047693,16.5269810830362,16.3180644072322,16.0407187777092,15.8867778740861,15.6640902495305,..
4,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,Central Europe and the Baltics,CEB,9.76067384088354,9.95849846965394,10.2076348610633,10.7450559900911,10.7825508634384,10.5442007818566,9.97322278608419,9.97663319700431,9.63832192953901,9.93466941518681,9.88846999346859,10.1346966663527,10.37061863823,10.1005806475465,9.77716690717954,..


In [17]:
pop_df = pd.DataFrame(
    columns = ["surr_year",
               "surr_country",
               "population_key",
              "birth_rate",
              "population_ages_0-14_female",
              "population_ages_0-14_male",
              "population_ages_15-64_female",
              "population_ages_15-64_male",
              "population_ages_65_and_above_female",
              "population_ages_65_and_above_male",
              "population_total",
              "population_growth",
              "mortality_rate_infant",
              "life_expectancy_at_birth_female",
              "life_expectancy_at_birth_male"]
)

In [18]:
country_list = ["Bangladesh", "Canada", "Chad", "Guinea", "Indonesia", "Mexico", "Philippines", "Togo", "United States"]
year_list = ["2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020"]

pop_country_dim_data = {}
for country in country_list:
    pop_country_dim_data[country]={}
    for year in year_list:
        pop_country_dim_data[country][year] = {}
counter = 0  

year_idx = {}
idx = 4
for year in year_list:
    year_idx[year] = idx
    idx=idx+1
    
for country in country_list:
    for year in year_list:
        pop_country_dim_data[country][year]["population_key"]= str(counter+6000)
        pop_country_dim_data[country][year]["surr_country"]= country
        pop_country_dim_data[country][year]["surr_year"]= year
        counter=counter+1
    
for row in pop_data.values.tolist():
    if row[2] in country_list:
        if row[0] == "Birth rate, crude (per 1,000 people)":
            populate_dim_data(pop_country_dim_data,row[2],"birth_rate",row)
        if row[0] == "Population ages 0-14, female (% of female population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_0-14_female",row)
        if row[0] == "Population ages 0-14, male (% of male population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_0-14_male",row)
        if row[0] == "Population ages 15-64, female (% of female population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_15-64_female",row)
        if row[0] == "Population ages 15-64, male (% of male population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_15-64_male",row)
        if row[0] == "Population ages 65 and above (% of total population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_65_and_above_female",row)
        if row[0] == "Population ages 65 and above, female (% of female population)":
            populate_dim_data(pop_country_dim_data,row[2],"population_ages_65_and_above_male",row)
        if row[0] == "Population, total":
            populate_dim_data(pop_country_dim_data,row[2],"population_total",row)
        if row[0] == "Population growth (annual %)":
            populate_dim_data(pop_country_dim_data,row[2],"population_growth",row)
        if row[0] == "Mortality rate, infant (per 1,000 live births)":
            populate_dim_data(pop_country_dim_data,row[2],"mortality_rate_infant",row)
        if row[0] == "Life expectancy at birth, female (years)":
            populate_dim_data(pop_country_dim_data,row[2],"life_expectancy_at_birth_female",row)
        if row[0] == "Life expectancy at birth, male (years)":
            populate_dim_data(pop_country_dim_data,row[2],"life_expectancy_at_birth_male",row)
    


for country in country_list:
    for year in year_list:
        pop_df = pop_df.append(pd.DataFrame([pop_country_dim_data[country][year]]), ignore_index=True)

In [19]:
pop_df

,surr_year,surr_country,population_key,birth_rate,population_ages_0-14_female,population_ages_0-14_male,population_ages_15-64_female,population_ages_15-64_male,population_ages_65_and_above_female,population_ages_65_and_above_male,population_total,population_growth,mortality_rate_infant,life_expectancy_at_birth_female,life_expectancy_at_birth_male
0,2005,Bangladesh,6000,24.053,34.5217030765266,34.214953122223,61.2429609022255,61.4159337686264,4.30386756246183,4.23533602124791,139035505,1.48474696408995,49.4,68.715,66.949
1,2006,Bangladesh,6001,23.384,34.0067893834996,33.8169062322044,61.6455255753353,61.7056455174323,4.41405087143509,4.34768504116512,140921154,1.34712103146787,47,69.24,67.318
2,2007,Bangladesh,6002,22.747,33.4752968728697,33.4035057012178,62.0681980541009,62.0170767921165,4.51926469056832,4.45650507302942,142660381,1.22663050489361,44.8,69.761,67.679
3,2008,Bangladesh,6003,22.153,32.9366297130732,32.9758067182168,62.505727436614,62.3517352758035,4.61617993332098,4.55764285031287,144304164,1.14564743804678,42.7,70.272,68.031
4,2009,Bangladesh,6004,21.605,32.3998269816199,32.5409763845715,62.9536947229883,62.7056022453165,4.70092139309264,4.64647829539186,145924795,1.11680643452794,40.7,70.773,68.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2016,United States,6139,12.2,18.4026326644684,19.6573709254641,65.051278947516,66.8537256836091,15.0336787501808,16.5460883880156,323071755,0.724676067451429,5.7,81.1,76.1
140,2017,United States,6140,11.8,18.2466676142417,19.4833737078997,64.8244160670525,66.6387612512319,15.4193658960742,16.9289163187058,325122128,0.63264399508256,5.7,81.1,76.1
141,2018,United States,6141,11.6,18.1048535943438,19.3259514439618,64.576790382131,66.4088716329403,15.8076540556035,17.3183560235252,326838199,0.526435395564053,5.6,81.2,76.2
142,2019,United States,6142,11.4,17.9519833351839,19.1591807905675,64.321850895079,66.17957827562,16.2096061428511,17.7261657697371,328329953,0.455381285963537,5.5,81.4,76.3


## Event

In [20]:
event_dimension_df = pd.DataFrame(
    columns=["Event_key", "Surrogate_year", "Surrogate_endMonth", "Surrogate_country", "Name", "Disaster_group", "Disaster_subgroup", "Disaster_type", "Start_date", "End_date", "Start_month", "End_month", "Total_deaths", "Num_injured"]
)

# CSV file of HNP country dimension statistics
raw_event_data = pd.read_csv('raw_data/event_data.csv', encoding='latin-1')
# Get rid of two columns we don't need
raw_event_data = raw_event_data.drop(
    ['ISO', 'Region', 'Location', 'Start Year', 'End Year'], axis=1
)
# Rename the columns for easier access
raw_event_data = raw_event_data.rename(
    columns={
        'Dis No': 'Id',
        'Disaster Group': 'Group',
        'Disaster Subgroup': 'Subgroup',
        'Disaster Type': 'Type',
        'Event Name': 'Name',
        'Start Month': 'sMonth',
        'Start Day': 'sDay',
        'End Month': 'eMonth',
        'End Day': 'eDay',
        'Total Deaths': 'Death',
        'No Injured': 'Injured'
    }
)

# Create a dictionary for surrogate month
dict_month = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
}


# Method to determine NaN value
def is_nan(x):
    return (x != x)

# Start key at 1000
x = 1000
col_id = raw_event_data.Id.tolist()
for y in col_id:
    event_row = {}
    event_row['Event_key'] = x
    x = x + 1
    event_row['Surrogate_year'] = raw_event_data.loc[(raw_event_data.Id == y), 'Year'].values[0]

    country = raw_event_data.loc[(raw_event_data.Id == y), 'Country'].values[0]
    if (country == 'United States of America (the)'):
        country = 'United States'
    elif (country == 'Philippines (the)'):
        country = 'Philippines'

    event_row['Surrogate_country'] = country
    event_row['Disaster_group'] = raw_event_data.loc[(raw_event_data.Id == y), 'Group'].values[0]
    event_row['Disaster_subgroup'] = raw_event_data.loc[(raw_event_data.Id == y), 'Subgroup'].values[0]
    event_row['Disaster_type'] = raw_event_data.loc[(raw_event_data.Id == y), 'Type'].values[0]
    event_row['Name'] = raw_event_data.loc[(raw_event_data.Id == y), 'Name'].values[0]
    event_row['Start_month'] = raw_event_data.loc[(raw_event_data.Id == y), 'sMonth'].values[0]
    tmp_mon = raw_event_data.loc[(raw_event_data.Id == y), 'eMonth'].values[0]
    event_row['End_month'] = tmp_mon
    event_row['Surrogate_endMonth'] = dict_month[tmp_mon]

    sd = raw_event_data.loc[(raw_event_data.Id == y), 'sDay'].values[0]
    if is_nan(sd) :
        event_row['Start_date'] = 1
    else:
        event_row['Start_date'] = sd

    ed = raw_event_data.loc[(raw_event_data.Id == y), 'eDay'].values[0]
    if is_nan(ed) :
        if event_row['Start_month'] == 1 | event_row['Start_month'] == 3 | event_row['Start_month'] == 5 | event_row['Start_month'] == 7 | event_row['Start_month'] == 8 | event_row['Start_month'] == 10 | event_row['Start_month'] == 12:
            event_row['End_date'] = 31
        elif event_row['Start_month'] == 4 | event_row['Start_month'] == 6 | event_row['Start_month'] == 9 | event_row['Start_month'] == 11:
            event_row['End_date'] = 30
        else:
            event_row['End_date'] = 28
    else:
        event_row['End_date'] = ed

    

    num_death = raw_event_data.loc[(raw_event_data.Id == y), 'Death'].values[0]
    if is_nan(num_death) :
        event_row['Total_deaths'] = 0
    else:
        event_row['Total_deaths'] = num_death
    
    num_injury = raw_event_data.loc[(raw_event_data.Id == y), 'Injured'].values[0]
    if is_nan(num_injury) :
        event_row['Num_injured'] = 0
    else:
        event_row['Num_injured'] = num_injury
    
    if pd.isnull(event_row['Name']):
        event_row['Name'] = str(event_row['Surrogate_year']) + " " + event_row['Surrogate_country'] + " " + event_row['Disaster_type']

    event_dimension_df = event_dimension_df.append(pd.DataFrame([event_row]), ignore_index=True)

# create null event
event_null_key = x
event_row['Event_key'] = event_null_key
event_row['Surrogate_year'] = 0
event_row['Surrogate_country'] = ''
event_row['Surrogate_endMonth'] = ''
event_row['Disaster_group'] = ''
event_row['Disaster_subgroup'] = ''
event_row['Disaster_type'] = ''
event_row['Name'] = ''
event_row['Start_month'] = 0
event_row['End_month'] = 0
event_row['Start_date'] = 0
event_row['End_date'] = 0
event_row['Total_deaths'] = 0
event_row['Num_injured'] = 0
event_dimension_df = event_dimension_df.append(pd.DataFrame([event_row]), ignore_index=True)

event_dimension_df

,Event_key,Surrogate_year,Surrogate_endMonth,Surrogate_country,Name,Disaster_group,Disaster_subgroup,Disaster_type,Start_date,End_date,Start_month,End_month,Total_deaths,Num_injured
0,1000,2005,March,Indonesia,2005 Indonesia Earthquake,Natural,Geophysical,Earthquake,28.0,28.0,3,3,915.0,1146.0
1,1001,2005,March,Bangladesh,2005 Bangladesh Storm,Natural,Meteorological,Storm,20.0,23.0,3,3,79.0,1500.0
2,1002,2005,February,Bangladesh,2005 Bangladesh Transport accident,Technological,Technological,Transport accident,19.0,19.0,2,2,149.0,0
3,1003,2005,January,Bangladesh,"Inter city express train ""Meghna""",Technological,Technological,Transport accident,16.0,16.0,1,1,0,200.0
4,1004,2005,January,Bangladesh,Garment Factory,Technological,Technological,Industrial accident,6.0,6.0,1,1,22.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,2519,2020,February,United States,2020 United States Flood,Natural,Hydrological,Flood,10.0,17.0,2,2,0,0
1520,2520,2020,June,United States,Tropical storm 'Cristobal',Natural,Meteorological,Storm,7.0,7.0,6,6,1.0,0
1521,2521,2020,August,United States,Hurricane 'Isaias',Natural,Meteorological,Storm,31.0,2.0,7,8,16.0,0
1522,2522,2020,October,United States,August Complex fire,Natural,Climatological,Wildfire,16.0,1.0,8,10,32.0,0


# Fact Table

In [21]:
fact_table_df = pd.DataFrame(
    columns = ["country_key","month_key","education_key", "health_key","qol_key","population_key"]
)


In [22]:
dict_qol = {
    'Bangladesh': 4,
    'Canada': 1,
    'Chad': 5,
    'Guinea': 5,
    'Indonesia': 3,
    'Mexico': 4,
    'Philippines': 3,
    'Togo': 5,
    'United States': 1
}

dict_di = {
    'Bangladesh': 2,
    'Canada': 1,
    'Chad': 3,
    'Guinea': 3,
    'Indonesia': 2,
    'Mexico': 1,
    'Philippines': 2,
    'Togo': 3,
    'United States': 1
}

dict_hdi = {
    'Bangladesh': 4,
    'Canada': 1,
    'Chad': 5,
    'Guinea': 5,
    'Indonesia': 3,
    'Mexico': 3,
    'Philippines': 3,
    'Togo': 4,
    'United States': 2
}


fact_table_data = {}
for country in country_list:
    fact_table_data[country]={}
    for year in year_list:
        fact_table_data[country][year] = {}
        for month in month_values:
            fact_table_data[country][year][month] = {}
for country in country_list:
    for year in year_list:
        for month in month_values:
            fact_table_data[country][year][month]["country"] = country
            fact_table_data[country][year][month]["year"] = year
            fact_table_data[country][year][month]["month"] = month
            fact_table_data[country][year][month]["qol"] = dict_qol[country]
            fact_table_data[country][year][month]["di"] = dict_di[country]
            fact_table_data[country][year][month]["hdi"] = dict_hdi[country]

for idx, row in country_dimension_df.iterrows():
    for month in month_values:
        fact_table_data[row.Name][str(row.Surrogate_year)][month]["country_key"] = row.Country_key
        
for country in country_list:
    for idx, row in month_df.iterrows():
        fact_table_data[country][str(row.Year)][row.Month]["month_key"] = row.Month_Key

for idx, row in education_df.iterrows():
    for month in month_values:
        fact_table_data[row.surr_country][row.surr_year][month]["education_key"] = row.education_key

for idx, row in health_df.iterrows():
    for month in month_values:
        fact_table_data[row.surr_country][row.surr_year][month]["health_key"] = row.health_key

for idx, row in qol_df.iterrows():
    for month in month_values:
        fact_table_data[row.surr_country][row.surr_year][month]["qol_key"] = row.qol_key

for idx, row in pop_df.iterrows():
    for month in month_values:
        fact_table_data[row.surr_country][row.surr_year][month]["population_key"] = row.population_key
        



for country in country_list:
    for year in year_list:
        for month in month_values:
            fact_table_df = fact_table_df.append(pd.DataFrame([fact_table_data[country][year][month]]), ignore_index=True)




In [23]:
events = event_dimension_df.drop(['Name','Disaster_group','Disaster_subgroup','Disaster_type','Start_date','End_date','Start_month','End_month','Total_deaths','Num_injured'], axis=1)
events['Surrogate_year'] = events['Surrogate_year'].astype(str)
fact_table = pd.merge(fact_table_df, events,  how='left', left_on=['year','country','month'], right_on = ['Surrogate_year','Surrogate_country','Surrogate_endMonth'])

for idx, row in fact_table.iterrows():
    if pd.isna(row.Event_key):
        fact_table.iat[idx,12] = 2523

fact_table

,country_key,month_key,education_key,health_key,qol_key,population_key,country,year,month,qol,di,hdi,Event_key,Surrogate_year,Surrogate_endMonth,Surrogate_country
0,1000,2001,3000,4000,5000,6000,Bangladesh,2005,January,4.0,2.0,4.0,1003,2005,January,Bangladesh
1,1000,2001,3000,4000,5000,6000,Bangladesh,2005,January,4.0,2.0,4.0,1004,2005,January,Bangladesh
2,1000,2002,3000,4000,5000,6000,Bangladesh,2005,February,4.0,2.0,4.0,1002,2005,February,Bangladesh
3,1000,2003,3000,4000,5000,6000,Bangladesh,2005,March,4.0,2.0,4.0,1001,2005,March,Bangladesh
4,1000,2003,3000,4000,5000,6000,Bangladesh,2005,March,4.0,2.0,4.0,1023,2005,March,Bangladesh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2441,1143,2190,3143,4143,5143,6143,United States,2020,October,1.0,1.0,2.0,2503,2020,October,United States
2442,1143,2190,3143,4143,5143,6143,United States,2020,October,1.0,1.0,2.0,2522,2020,October,United States
2443,1143,2191,3143,4143,5143,6143,United States,2020,November,1.0,1.0,2.0,2490,2020,November,United States
2444,1143,2191,3143,4143,5143,6143,United States,2020,November,1.0,1.0,2.0,2491,2020,November,United States


In [24]:
# Removing surrogate keys
country_dimension_df = country_dimension_df.drop(['Surrogate_year'], axis=1)
education_df = education_df.drop(['surr_year', 'surr_country'], axis=1)
health_df = health_df.drop(['surr_year', 'surr_country'], axis=1)
qol_df = qol_df.drop(["surr_country","surr_year"], axis=1)
pop_df = pop_df.drop(['surr_year', 'surr_country'], axis=1)
fact_table = fact_table.drop(['Surrogate_year','Surrogate_country','year','country','month','Surrogate_endMonth'], axis=1)
event_dimension_df = event_dimension_df.drop(['Surrogate_year','Surrogate_country','Surrogate_endMonth'], axis=1)
# Creating csv of clean data for data mart
country_dimension_df.to_csv('clean_data/country.csv', index=False)
month_df.to_csv('clean_data/month.csv', index=False)
education_df.to_csv('clean_data/education.csv', index=False)
health_df.to_csv('clean_data/health.csv', index=False)
qol_df.to_csv('clean_data/quality_of_life.csv', index=False)
pop_df.to_csv('clean_data/population.csv', index=False)
event_dimension_df.to_csv('clean_data/event.csv', index=False)
fact_table.to_csv('clean_data/fact_table.csv', index=False)

In [25]:
fact_table.shape[0]

2446